In [ ]:
import plotly.express as px
import numpy as np
import cv2
import os

In [ ]:
# constants
GRID_SIZE = (8,11)
SQUARE_SIZE = 11

# getting the images path
images_path = "./images_and_poses_for_project_assignment/"
images_path = [os.path.join(images_path, imagename) for imagename in os.listdir(images_path) if imagename.endswith(".png")]

In [ ]:
filepath = images_path[5]
image = cv2.imread(filepath)

# fig = px.imshow(image)
# fig.show()

In [ ]:
return_value, corners = cv2.findChessboardCorners(image, patternSize=GRID_SIZE)
corners=corners.reshape((88,2)).copy()
corners[-1,:]

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 100, 0.001) # tuple for specifying the termination criteria of the iterative refinement procedure cornerSubPix()
cv2.cornerSubPix(gray,corners,(5,5),(-1,-1),criteria)
corners[-1,:] # last row of corners after refinement. Compare the refined coordinates to the original ones in the output above

In [ ]:
image_copy = image.copy()
cv2.drawChessboardCorners(image_copy, GRID_SIZE, corners, return_value)
fig = px.imshow(cv2.cvtColor(image_copy,cv2.COLOR_BGR2RGB))
fig.show()

In [ ]:
another_copy=image.copy()

real_coordinates = np.empty_like(corners)

for index, corner in enumerate(corners):

    u_coord = corner[0] #pxl
    v_coord = corner[1]

    # unravel_index converts an index from linear to 2d
    # example: index 7 of a 3x4 matrix corresponds to position (1, 3) of the matrix (NOTE: 0-based indexing)
    # notice that we revert the grid size as 2d matrices are stored differently in Python and the OpenCV backend
    grid_size_cv2 = tuple(reversed(GRID_SIZE))
    u_index, v_index = np.unravel_index(index, grid_size_cv2)

    # the coordinates of the corner w.r.t. the reference corner at position (0,0) of the corners array
    x_mm = (u_index) * SQUARE_SIZE
    y_mm = (v_index) * SQUARE_SIZE

    # we store them in the real_coordinates array
    real_coordinates[index,:] = [x_mm, y_mm]

    # this superimposes the text to the image - NOTE: it overwrites current image, this is why we work on a copy of img

    cv2.putText(another_copy, text=f"{x_mm};{y_mm}", org=(round(u_coord),round(v_coord)),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=.4, color=(255,0,0), thickness=1)

fig = px.imshow(another_copy)
fig.show()

In [ ]:
# CONSTRUCT A
A = np.empty((0, 9), dtype=float)
b = np.empty((0, 1), dtype=float)

for index, corner in enumerate(corners):
    Xpixel = corners[index, 0]
    Ypixel = corners[index, 1]
    Xmm = real_coordinates[index, 0]
    Ymm = real_coordinates[index, 1]

    m = np.array([Xmm, Ymm, 1]).reshape(1, 3)
    O = np.array([0, 0, 0]).reshape(1, 3)

    # Construct A and b
    A = np.vstack((A, np.hstack((m, O, -Xpixel * m))))
    A = np.vstack((A, np.hstack((O, m, -Ypixel * m))))
    b = np.vstack((b, np.array([[0], [0]])))

In [ ]:
U, S, Vh = np.linalg.svd(A)
h = Vh.transpose()[:, -1] # keep the last column of V (pay attention to the transposition)
# print(h)
H = h.reshape(3, 3) # reshape from 9 to 3x3
H # this is the estimated homography

In [ ]:
def homography(image_path, GRID_SIZE, SQUARE_SIZE):
    image = cv2.imread(image_path)
    _, corners = cv2.findChessboardCorners(image, patternSize=GRID_SIZE)
    corners = corners.reshape((88,2)).copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 100, 0.001)
    cv2.cornerSubPix(gray,corners,(5,5),(-1,-1),criteria)
    real_coordinates = np.empty_like(corners)

    for index, corner in enumerate(corners):
        # unravel_index converts an index from linear to 2d
        # example: index 7 of a 3x4 matrix corresponds to position (1, 3) of the matrix (NOTE: 0-based indexing)
        # notice that we revert the grid size as 2d matrices are stored differently in Python and the OpenCV backend
        grid_size_cv2 = tuple(reversed(GRID_SIZE))
        u_index, v_index = np.unravel_index(index, grid_size_cv2)

        # the coordinates of the corner w.r.t. the reference corner at position (0,0) of the corners array
        x_mm = (u_index) * SQUARE_SIZE
        y_mm = (v_index) * SQUARE_SIZE

        # we store them in the real_coordinates array
        real_coordinates[index,:] = [x_mm, y_mm]

    A = np.empty((0, 9), dtype=float)
    b = np.empty((0, 1), dtype=float)

    for index, corner in enumerate(corners):
        Xpixel = corners[index, 0]
        Ypixel = corners[index, 1]
        Xmm = real_coordinates[index, 0]
        Ymm = real_coordinates[index, 1]

        m = np.array([Xmm, Ymm, 1]).reshape(1, 3)
        O = np.array([0, 0, 0]).reshape(1, 3)

        # Construct A and b
        A = np.vstack((A, np.hstack((m, O, -Xpixel * m))))
        A = np.vstack((A, np.hstack((O, m, -Ypixel * m))))
        b = np.vstack((b, np.array([[0], [0]])))

    U, S, Vh = np.linalg.svd(A)
    h = Vh.transpose()[:, -1] # keep the last column of V (pay attention to the transposition)
    H = h.reshape(3, 3) # reshape from 9 to 3x3

    return H

In [ ]:
homography(filepath, GRID_SIZE, SQUARE_SIZE)

In [ ]:
print(H.T[0][0])
print(H.T[0,0])

In [ ]:
def v_ij(H, i, j):
    h = H.T
    return np.array([
        h[i][0]*h[j][0],
        h[i][0]*h[j][1] + h[i][1]*h[j][0],
        h[i][1]*h[j][1],
        h[i][2]*h[j][0] + h[i][0]*h[j][2],
        h[i][2]*h[j][1] + h[i][1]*h[j][2],
        h[i][2]*h[j][2]
    ])

In [ ]:
def build_V(homographies):
    V = []
    for H in homographies:
        V.append(v_ij(H, 0, 1))
        V.append(v_ij(H, 0, 0) - v_ij(H, 1, 1))
    return np.array(V)

In [ ]:
Hs = []

for path in images_path:
    Hs.append(homography(path, GRID_SIZE, SQUARE_SIZE))

V = build_V(Hs)
_, _, vh = np.linalg.svd(V)
b = vh[-1]
B11, B12, B22, B13, B23, B33 = b

v0 = (B12*B13 - B11*B23) / (B11*B22 - B12**2)
lambda_ = B33 - (B13**2 + v0*(B12*B13 - B11*B23)) / B11

fx = np.sqrt(lambda_ / B11)
fy = np.sqrt(lambda_ * B11 / (B11*B22 - B12**2))
skew = -B12 * fx**2 * fy / lambda_
cx = skew * v0 / fy - B13 * fx**2 / lambda_
cy = v0

K = np.array([
    [fx, skew, cx],
    [0,  fy,   cy],
    [0,  0,    1]
])

In [ ]:
print(K)

In [ ]:
objp = np.zeros((8*11, 3), np.float32)
objp[:, :2] = np.mgrid[0:8, 0:11].T.reshape(-1, 2)
objp *= SQUARE_SIZE * 0.001

In [ ]:
# objpoints = []
# imgpoints = []

# # objp defined once (object points)

# for path in images_path:
#     img = cv2.imread(path)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     ret, corners = cv2.findChessboardCorners(gray, patternSize=GRID_SIZE)

#     if ret:
#         objpoints.append(objp)
#         imgpoints.append(corners)

# image_size = gray.shape[:2][::-1]

# ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
#     objpoints,
#     imgpoints,
#     image_size,
#     None,
#     None
# )

# print(K)

# Transposition

In [ ]:
rect_width = 99     # [mm]
rect_height = 44    # [mm]

bottom_left_corner_x = 11   # [mm]
bottom_left_corner_y = 33   # [mm]

In [ ]:
# FROM THE LAB
vx = bottom_left_corner_x + np.array([0, 0, rect_width, rect_width])
vy = bottom_left_corner_y + np.array([0, rect_height, rect_height, 0])

homogeneousIN = np.vstack((vx, vy, np.ones_like(vx)))
homogeneousOUT = H @ homogeneousIN

xyOUT = (homogeneousOUT / homogeneousOUT[2])[:2]
xyOUT

In [ ]:
cv2.polylines(image, np.int32([xyOUT.transpose()]), isClosed=True, color=(255, 0, 0), thickness=4)
# plt.figure(figsize = (6.4*2, 4.8*2)) # default is (6.4, 4.8) inches
# plt.imshow(image)
fig = px.imshow(image)
fig.show()